# Hazardous Waste Prediction

#### Input the rail car parameters below, and run all following cells for result

In [1]:
## INPUTS ##
origin = 'New Orleans, LA-MS'
destination = 'Birmingham, AL'
interchange_state_1 = 'AL'
interchange_state_2 = 'None'
interchange_state_3 = 'None'
car_ownership_code = 'P' # P, R, T
intermodal_code = 1 # 1, 2, 9
shortline_miles = 350
number_of_articulated_units = 0

---

In [2]:
import plaidml.keras as pk
pk.install_backend()

import numpy as np
import pandas as pd
import pickle
from keras.models import load_model

In [3]:
with open('encoder.2.pickle', 'rb') as f:
    encoder = pickle.load(f)
    
with open('min_max_scaler.pickle', 'rb') as f:
    scaler = pickle.load(f)
    
model = load_model('simple_cnn.1.h5')

INFO:plaidml:Opening device "metal_amd_radeon_pro_570x.0"


In [4]:
#origin = origin.replace(', ', '').replace(' ', '').replace('-', '')
#destination = destination.replace(', ', '').replace(' ', '').replace('-', '')

data_dict = {
    'estimated_short_line_miles': [shortline_miles], 
    'number_of_articulated_units': [number_of_articulated_units],
    'car_ownership_category_code': [car_ownership_code],
    'all_rail_intermodal_code': [intermodal_code],
    'origin_location': [origin],
    'interchange_state_1': [interchange_state_1],
    'interchange_state_2': [interchange_state_2],
    'interchange_state_3': [interchange_state_3],
    'terminal_location': [destination]
}

df = pd.DataFrame(data_dict)

num_cols = [
    'estimated_short_line_miles',
    'number_of_articulated_units'
]

cat_cols = [
    'car_ownership_category_code', 
    'all_rail_intermodal_code',
    'origin_location',
    'interchange_state_1',
    'interchange_state_2',
    'interchange_state_3',
    'terminal_location'
]

encoded_cats = encoder.transform(df[cat_cols])
scaled_nums = scaler.transform(df[num_cols])

from scipy.sparse import hstack, csr_matrix
input_data = hstack((scaled_nums, encoded_cats))
input_data = csr_matrix(input_data).toarray()
input_data = input_data.reshape(input_data.shape[0], input_data.shape[1], 1)

In [6]:
prob = model.predict(input_data)[0][0]

---

## Result

In [10]:
print(f'There is a {round(prob*100,2)}% chance of this freight containing hazardous materials.')

There is a 99.95% chance of this freight containing hazardous materials.


*this result is produced by our simple 1D CNN model with one-hot encoded routes*